# Battle of the neighbourhoods

### Introduction

The purpose of this project is the find out the distribution of different types of restaurants in different ares in Chennai using Foursquare API. Chennai is home to a lot of traditional south indian delicacies. The local food is very good. There is alot of continental & oriental varieties also available which makes Chennai a must visit forfoodies and home to a lot of delicious food. The purpose of this project is to identify the spread of different cuisines of food in different localities. Of course it is not possible to map the street food since those are not listed in the Foursquare site.

### Data

The data for mapping is obtained as follows. District of Ankara are obtained from https://www.mapsofindia.com/pincode/india/tamil-nadu/chennai/ Latitude and Longitude values are obtained by using "geocoder". All data related to locations will be obtained by using FourSquare API and Python Libraries.

In [3]:
# Import libraries - long wait to sort the environment
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests for postal codes in wikipedia
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup


print('Libraries imported!')

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
done

# All requested packages already installed.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |         

In [5]:
url = "https://en.wikipedia.org/wiki/Areas_of_Chennai"
page = requests.get(url).text
soup = BeautifulSoup(page,"html5lib")
table = soup.find('table')
rows=table.tbody.findAll("tr")
len(table)

#cols

2

In [7]:
df = pd.DataFrame(columns=["Location", "Pincode"])

for row in rows:
    col=row.find_all("td")
    if (col != []):
        Location = col[0].text
        
        
        df = df.append({"Location":Location}, ignore_index=True)

df=df[1:]
df.head()

AttributeError: 'NoneType' object has no attribute 'items'

            Location Pincode
1            Adyar\n     NaN
2          Alandur\n     NaN
3        Alapakkam\n     NaN
4         Alwarpet\n     NaN
5  Alwarthirunagar\n     NaN

In [8]:
len(df)

162

In [9]:
pip install geocoder

     |████████████████████████████████| 98 kB 6.8 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_user_agent")

In [11]:
df["loc"] = df["Location"].apply(geolocator.geocode)
df["point"]= df["loc"].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat', 'lon', 'altitude']] = pd.DataFrame(df['point'].to_list(), index=df.index)
df.drop(['loc', 'point', 'altitude'], axis=1, inplace=True)

In [12]:
df.rename(columns = {'lat':'Latitude', 'lon':'Longitude'}, inplace = True)
df = df[df['Latitude'].notna()]
df

AttributeError: 'NoneType' object has no attribute 'items'

                        Location Pincode   Latitude   Longitude
1                        Adyar\n     NaN  13.006450   80.257779
2                      Alandur\n     NaN  13.002822   80.171919
3                    Alapakkam\n     NaN  13.049901   80.165435
4                     Alwarpet\n     NaN  13.033860   80.254549
5              Alwarthirunagar\n     NaN  13.047486   80.183669
6                     Ambattur\n     NaN  13.112886   80.159862
7                  Aminjikarai\n     NaN  13.072140   80.220545
8                   Anna Nagar\n     NaN  13.087200   80.216442
9                      Annanur\n     NaN  13.116356   80.126913
10                  Arumbakkam\n     NaN  13.074371   80.208131
11                 Ashok Nagar\n     NaN  13.040073   80.215925
12                       Avadi\n     NaN  13.125476   80.094090
13                  Ayanavaram\n     NaN  13.094616   80.235410
14                Besant Nagar\n     NaN  12.999691   80.268226
15                Basin Bridge\n     NaN

In [13]:
address = 'Chennai, India'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The decimal coordinates of CHennai are {}, {}.'.format(latitude, longitude))

The decimal coordinates of CHennai are 13.0836939, 80.270186.


In [14]:
# create map of Bogotá using latitude and longitude values
map_chennai = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, local in zip(df['Latitude'], df['Longitude'], df['Location']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_chennai)  
    
map_chennai